In [1]:
from json import dumps, loads
import time
import unittest
import requests
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient


In [2]:
    
G = setup()[0]
(provider_priv1, provider_pub1) = key_gen(setup())
(provider_priv2, provider_pub2) = key_gen(setup())
(provider_priv3, provider_pub3) = key_gen(setup())
client1 = ChainspaceClient(port=5000)
client2 = ChainspaceClient(port=5000)
client3 = ChainspaceClient(port=5000)


In [7]:
init_transaction = surge.init()
init_token1 = init_transaction['transaction']['outputs'][0]
init_token2 = init_transaction['transaction']['outputs'][1]
init_token3 = init_transaction['transaction']['outputs'][2]
client1.process_transaction(init_transaction)

inputs: ()
reference_inputs: ()


In [ ]:
create_surge_client_txn = surge.create_surge_client(
    (init_token1,),
    None,
    (pack(provider_pub1),),
    pack(provider_priv1),
)
surge_client1 = create_surge_client_txn['transaction']['outputs'][0]
ebtoken1 = create_surge_client_txn['transaction']['outputs'][2]
client1.process_transaction(create_surge_client_txn)

In [ ]:
create_surge_client_txn = surge.create_surge_client(
    (init_token2,),
    None,
    (pack(provider_pub2),),
    pack(provider_priv2),
)
surge_client2 = create_surge_client_txn['transaction']['outputs'][0]
ebtoken2 = create_surge_client_txn['transaction']['outputs'][2]
client2.process_transaction(create_surge_client_txn)

In [ ]:
create_surge_client_txn = surge.create_surge_client(
    (init_token3,),
    None,
    (pack(provider_pub3),),
    pack(provider_priv3),
)
surge_client3 = create_surge_client_txn['transaction']['outputs'][0]
ebtoken3 = create_surge_client_txn['transaction']['outputs'][2]
client3.process_transaction(create_surge_client_txn)

In [ ]:
submit_bid_txn = surge.submit_bid(
    (ebtoken1,),
    None,
    ('EBBuy',50,),
    pack(provider_priv1),
)
bid1 = submit_bid_txn['transaction']['outputs'][0]
ebtoken1 = submit_bid_txn['transaction']['outputs'][1]
client1.process_transaction(submit_bid_txn)

In [ ]:
submit_bid_txn = surge.submit_bid(
    (ebtoken2,),
    None,
    ('EBSell',55,),
    pack(provider_priv2),
)
bid2 = submit_bid_txn['transaction']['outputs'][0]
ebtoken2 = submit_bid_txn['transaction']['outputs'][1]
client1.process_transaction(submit_bid_txn)

In [6]:
objs = client1.get_objects({'status':1})
type(objs[0])

http://127.0.0.1:5000/api/1.0/objects?status=1
HTTP/1.1 200 OK
[{u'status': u'active', u'id': u'1_d0a3ee72fbbe2ce4dd31e4f9fecc081ec3fa9af0abda50dd9463d4ed0539de1e', u'value': {u'type': u'InitToken', u'location': 1}}, {u'status': u'active', u'id': u'2_d84363f405bf730b14a01f7e8cf07feeeac35a3b2ac62582e022c2629f135d12', u'value': {u'type': u'InitToken', u'location': 2}}, {u'status': u'active', u'id': u'0_32d1498f29d15eee49aeb95c00033500c6cb1f4869a1e9114937ec44031d9280', u'value': {u'type': u'SurgeClient', u'location': 0, u'pub': u'c7220292cd02c9bd0378ac85ae7886818ccdd678618f6d988613399843c3f607d0e5821dd4'}}, {u'status': u'active', u'id': u'0_b34cfcab92b006949f1521c0119b48992ee9e64c624516689ae0c17fff4cb5e4', u'value': {u'type': u'VoteSlipToken', u'location': 0, u'pub': u'c7220292cd02c9bd0378ac85ae7886818ccdd678618f6d988613399843c3f607d0e5821dd4'}}, {u'status': u'active', u'id': u'0_07eb24aa0c4ab01390172aa2d92c27fd505fad76012ddfda97cadddc3ad8fbad', u'value': {u'type': u'EBBuy', u'location': 

chainspacecontract.contract.ChainspaceObject